In [1]:
import numpy as np
import pandas as pd
import import_ipynb
import glob
import copy
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [2]:
countries={ 'Austria': 'AT', 'Belgium': 'BE',  'Bulgaria': 'BG', 'Switzerland': 'CH', 'Czech Republic': 'CZ',  'Germany': 'DE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR',  'Greece': 'GR', 'Hungary': 'HU', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Latvia': 'LV', 'Montenegro': 'ME','Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'UK'}

abbr_list=list(countries.values())

In [ ]:
fuels=['Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other', 'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore']

In [ ]:
import matplotlib.pyplot as plt 

# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

In [3]:
import internal_inconsistencies as ii
import random

importing Jupyter notebook from internal_inconsistencies.ipynb
importing Jupyter notebook from data_preperation.ipynb


## Check total missing values

## Fill missing values

In [60]:
x = [np.nan,np.nan,9]
if not all(i is np.nan for i in x):
    print('all none')
y = np.nanmean(x)
x = [y if pd.isna(i) else i for i in x]
print(x)
print(np.mean(np.array(x)))

all none
[9.0, 9.0, 9]
9.0


In [ ]:
selected_index = 1381
selected_values = [[1375,1798],[1377,1935],[1379,1940],[1383,np.nan],[1385,1933,],[1387,1943]]
polynomial(selected_values, selected_index)
x_values = [i[0] for i in selected_values]
y_v = [i[1] for i in selected_values]
y_value = -978121618.0074843 + x**3*(2.12179738e+06) + x**2*(-1.53423437e+03) + x*(3.69791667e-01)
# plt.plot(x_values, y_value)


In [71]:
country = 'Lithuania'
df,length = ii.omit_dst(pd.read_csv(f'../Data Sources/ENTSO-E/2018/Load/{country}.csv', low_memory=False).iloc[:, 2:])
df, counter = fill_missing_data(df, length)


In [70]:
def polynomial(selected_values, selected_index):

    # We get all the indexes in the 'selected_values' to a 2D numpy array 'X'.
    # We get all the values in the 'selected_values'to a 1D numpy array 'y'
    # Then we fill the null values in array 'y' with the mean value of the array.
    # Then we divide 'X' and 'y' values in the ratio of 30% test and 70% train data.
    # We create an array of degree values from 1 to 10.
    # Then we iterate the 'degrees' one by one and create polynomial values of 'x_train' data called 'x_poly_train' based on the value of the degree
    # Then we fit the polynomial linear regression function using 'x_poly_train' data and 'y_train' data.
    # Then based on the polynomial function, using the 'x_poly_test' data we predict the values of the 'y_test' data
    # Then based on the predicted values and 'y_test" data we calculate the Root Mean Square Error.
    # Applying the last 4 steps for each degree value, we select the degree value which gives the Lowest Root Mean Square Error.
    # Then we fit the polynomial linear regression function again using that degree which gives the Lowest Root Mean Square Error.
    # Based on the polynimial function we get the predicted value of the null value.

    X = np.array([i[0] for i in selected_values]
                 ).reshape(len(selected_values), 1)
    y = [i[1] for i in selected_values]
    mean = np.nanmean(y)
    y = [mean if pd.isna(x) else x for x in y]

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

    degrees = np.arange(1, 11)
    min_rmse, min_deg = 1e10, 0

    for degree in degrees:

        # Preparing polynomial Train features based on x_train
        poly_features = PolynomialFeatures(degree=degree, include_bias=False)
        x_poly_train = poly_features.fit_transform(x_train)

        # Polynomial linear regression based on train data
        poly_reg = LinearRegression()
        poly_reg.fit(x_poly_train, y_train)

        # Predicting y values and getting root mean squared error based on predicted y values and y_test values
        x_poly_test = poly_features.fit_transform(x_test)
        poly_predict = poly_reg.predict(x_poly_test)
        poly_mse = mean_squared_error(y_test, poly_predict)
        poly_rmse = np.sqrt(poly_mse)

        # Selecting the best degree of the polynimial function based on lowest root mean squared error
        if min_rmse > poly_rmse:
            min_rmse = poly_rmse
            min_deg = degree

    # Fitting the regression function again based on the selected best degree above
    poly_features = PolynomialFeatures(degree=min_deg, include_bias=False)
    x_poly_train = poly_features.fit_transform(x_train)
    poly_reg = LinearRegression()
    poly_reg.fit(x_poly_train, y_train)

    prediction = poly_reg.predict(
        poly_features.fit_transform([[selected_index]]))[0]
    if prediction < 0:
        prediction = 0

    return(prediction)


def mean(selected_values):
    if pd.isnull(selected_values).sum() != len(selected_values):
        mean_value = np.nanmean(selected_values)
        selected_values = [mean_value if pd.isna(x) else x for x in selected_values]
        prediction = np.mean(np.array(selected_values))
    else:
        prediction = 0

    return prediction


In [68]:
selected_values


[23071.0, 22305.0, 20971.0]

In [69]:
def fill_missing_data(df,length):

    # 1. In the following section, we get the indexes and values of a column of a dataframe to a dictionary called 'column_data'.
    # 2. Then we iterate the "column_data" dictionary line by line untill a null value is found (We called this index as 'selected_index').
    # 3. If the index of the value is within the range of after first 3 hours and before the last 3 hours of the column we create a empty list called 'selected_values'.
    # 4. Else if the index is null but it is not in the above range, we get the mean value of first 3 hours or mean value of last 3 hours according to the position of the index of the null value as the missing null value.
    # 5. In 3, We append the 'seleced_values' list with the indexes of the 3 hours before the index of the null value and 3 hours after the index of the null value and respective values of those indexes.
    # 6. In the 6 values of this list if more than 3 are null values and the 'selected_index' is within the range of after first 27 hours and before the last 27 hours of the column we create another empty list called 'selected_values'.
    # 7. Else if more than 3 are null values but not in the range mentioned above, get the mean value of the remaining values in the 'seleced_values' list as the missing null value.
    # 8. If both the two conditions in 6 and 7 are not met, get the missing null value by using the polynomial function.
    # 9. In 6, We append the 'seleced_values' list with the indexes of the (1,2,3,21,22,23,25,26,27) hours before and after the index of the null value and respective values of those indexes.
    # 10. In the 18 values of this llist, if more than 12 are null values and the 'selected_index' is within the range of after first 51 hours and before the last 51 hours of the column we create another empty list called 'selected_values'.
    # 11. Else if more than 12 are null values but not in the range mentioned above, get the mean value of the remaining values in the 'seleced_values' list as the missing null value.
    # 12. If both the two conditions in 10 and 11 are not met, get the missing null value by using the polynomial function.
    # 13. In 10, We append the 'seleced_values' list with the indexes of the (1,2,3,21,22,23,25,26,27,45,46,47,49,50,51) hours before and after the index of the null value and respective values of those indexes.
    # 14. In the 18 values of this llist, if more than 24 are null values and but not all 30 are null values, get the mean value of the remaining values in the 'seleced_values' list as the missing null value.
    # 15. If all 30 are null values, make the missing null value 0.
    # 16. Then we replace 'n/e' and any remining null values with 0.

    counter =0  
    for column in df.columns.values:
        column_data = {}
        for index, value in enumerate(df.loc[:, column]):
            column_data[index] = value

        for selected_index, selected_value in column_data.items():

            if pd.isnull(column_data[selected_index]) and selected_index in range(3*length, len(df[column])-3*length):
                selected_values = []
                for i in [x for x in range(-3,4) if x!=0]:
                    selected_values.append([selected_index + i*length,column_data[selected_index + i*length]])
                if pd.isnull(selected_values).sum() >=3 and selected_index in range(27*length, len(df[column])-27*length):
                    selected_values = []
                    for i in [x for x in range(-3, 4) if x != 0]:
                        for j in [-24,0,24]:
                            selected_values.append([selected_index + (i+j)*length,column_data[selected_index + (i+j)*length]])
                    if pd.isnull(selected_values).sum() >= 12 and selected_index in range(51*length, len(df[column])-51*length):
                        selected_values = []
                        for i in [x for x in range(-3, 4) if x != 0]:
                            for j in [-48,-24,0, 24,48]:
                                selected_values.append([selected_index + (i+j)*length,column_data[selected_index + (i+j)*length]])
                        # print(selected_values)
                        if pd.isnull(selected_values).sum() >= 24 and pd.isnull(selected_values).sum() < len(selected_values):
                            print([i[1] for i in selected_values])
                            prediction = mean([i[1] for i in selected_values])
                            df.loc[selected_index,column] = prediction
                            counter +=1

                        elif pd.isnull(selected_values).sum() < 24:
                            prediction = polynomial(selected_values,selected_index)
                            df.loc[selected_index, column] = prediction
                            counter += 1

                        else:
                            df.loc[selected_index, column] = 0
                            counter += 1

                    elif pd.isnull(selected_values).sum() >= 12:
                        prediction = mean([i[1] for i in selected_values])
                        df.loc[selected_index, column] = prediction
                        counter += 1

                    else:
                        prediction = polynomial(selected_values,selected_index)
                        df.loc[selected_index, column] = prediction
                        counter += 1

                elif pd.isnull(selected_values).sum() >= 3:
                    prediction = mean([i[1] for i in selected_values])
                    df.loc[selected_index, column] = prediction
                    counter += 1

                else:
                    prediction = polynomial(selected_values, selected_index)
                    df.loc[selected_index, column] = prediction
                    counter += 1

            elif pd.isnull(column_data[selected_index]) and selected_index < 3*length:
                selected_values =  [column_data[i] for i in range(3*length)]
                prediction = mean(selected_values)
                df.loc[selected_index, column] = prediction
                counter += 1

            elif pd.isnull(column_data[selected_index]) and selected_index >= (len(df[column])-3*length): 
                selected_values = [column_data[i] for i in range(len(df[column])-3*length,len(df[column]))]
                prediction = mean(selected_values)
                df.loc[selected_index, column] = prediction
                counter += 1

    # df = df.replace(['n/e', np.nan], 0)
    
    # Following command creates a numpy array with a length similar to the length of the dataframe.
    # Values of the array are obtained by getting the floor division of the length value.
    # For example, when divider=4, this array will be [0,0,0,0,1,1,1,1,2,2,2,2,....].
    # Then the rows of the dataframe will be grouped according to the order of the numpy array with the mean value of those 4 rows.
    # For example, in the numpy array first 4 values are similar. Accordingly first 4 rows of the dataframe will be grouped and get the mean value of those rows

    df = df.groupby(np.arange(len(df))//length).mean()
    print(df.isnull().sum().sum())
    return df,counter


## DO NOT DELETE

In [29]:
# x = ['Germany', 'Spain', 'Greece', 'Hungary', 'Montenegro']
for i in range(9):
    globals()[f'counter_{i}'] =0
counter =0
f = open('neighbors of missing values.txt', 'w')
# for country,abbr in countries.items():
country='Italy'
df, length = ii.omit_dst(pd.read_csv(f'../Data Sources/ENTSO-E/2018/Load/{country}.csv', low_memory=False).iloc[:,2:])
if 'Hydro Pumped Storage  - Actual Consumption [MW]' in df.columns.values:
    df=df.drop(['Hydro Pumped Storage  - Actual Consumption [MW]'],axis=1)
for column in df.columns.values:
    column_data = {}
    for index, value in enumerate(df.loc[:, column]):
        column_data[index] = value
    # null_values = [index for index,value in enumerate(df.loc[:,column]) if pd.isnull(df.loc[index,column])] #NEW

    for selected_index, selected_value in column_data.items():
        if pd.isnull(column_data[selected_index]) and selected_index in range(3*length, len(df[column])-3*length):
    
    # for selected_index in null_values:
    #     if selected_index in range(3*length, len(df[column])-3*length):
            selected_values = []
            for i in [x for x in range(-3,4) if x!=0]:
                selected_values.append([selected_index + i*length,column_data[selected_index + i*length]]) # Change df.loc with column_data
            if pd.isnull(selected_values).sum() >=3 and selected_index in range(27*length, len(df[column])-27*length):
                selected_values = []
                for i in [x for x in range(-3, 4) if x != 0]:
                    for j in [-24,0,24]:
                        selected_values.append([selected_index + (i+j)*length,column_data[selected_index + (i+j)*length]])
                if pd.isnull(selected_values).sum() >= 12 and selected_index in range(51*length, len(df[column])-51*length):
                    selected_values = []
                    for i in [x for x in range(-3, 4) if x != 0]:
                        for j in [-48,-24,0, 24,48]:
                            selected_values.append([selected_index + (i+j)*length,column_data[selected_index + (i+j)*length]])
                    # print(selected_values)
                    if pd.isnull(selected_values).sum() >= 24 and pd.isnull(selected_values).sum() < len(selected_values):
                        prediction = np.mean(np.array([i[1] for i in selected_values]))
                        f.write(f'{country},{column},{selected_index},{len(selected_values)},{len(selected_values) - pd.isnull(selected_values).sum()},{prediction}, Average\n')
                        counter_0 +=1
                    elif pd.isnull(selected_values).sum() < 24:
                        min_deg,prediction = polynomial(selected_values,selected_index)
                        f.write(f'{country},{column},{selected_index},{len(selected_values)},{len(selected_values) - pd.isnull(selected_values).sum()},{prediction}, PLR, {min_deg}\n')
                        counter_1 +=1
                    else:
                        f.write(f'{country},{column},{selected_index},{len(selected_values)},{len(selected_values) - pd.isnull(selected_values).sum()},{0}, Zero\n')
                        counter_2 +=1
                elif pd.isnull(selected_values).sum() >= 12:
                    prediction = np.mean(np.array([i[1] for i in selected_values]))
                    f.write(f'{country},{column},{selected_index},{len(selected_values)},{len(selected_values) - pd.isnull(selected_values).sum()},{prediction}, Average\n')
                    counter_3 +=1
                else:
                    min_deg,prediction = polynomial(selected_values,selected_index)
                    f.write(f'{country},{column},{selected_index},{len(selected_values)},{len(selected_values) - pd.isnull(selected_values).sum()},{prediction}, PLR, {min_deg}\n')
                    counter_4 +=1
            elif pd.isnull(selected_values).sum() >= 3:
                prediction = np.mean(np.array([i[1] for i in selected_values]))
                f.write(f'{country},{column},{selected_index},{len(selected_values)},{len(selected_values) - pd.isnull(selected_values).sum()},{prediction}, Average\n')
                counter_5 += 1
            else:
                min_deg,prediction = polynomial(selected_values, selected_index)
                f.write(f'{country},{column},{selected_index},{len(selected_values)},{len(selected_values) - pd.isnull(selected_values).sum()},{prediction}, PLR, {min_deg}\n')
                counter_6 +=1
        elif pd.isnull(column_data[selected_index]) and selected_index < 3*length: # Add pd.isnull(df.loc[selected_index]) and to the begining
            prediction = np.mean(np.array([column_data[i] for i in range(3*length)])) # Chnage df.loc with column_data[i]
            f.write(f'{country},{column},{selected_index},{prediction}, Average\n')
            counter_7 +=1
        elif pd.isnull(column_data[selected_index]) and selected_index >= (len(df[column])-3*length): # Add pd.isnull(df.loc[selected_index]) and to the begining
            prediction = np.mean(np.array([column_data[i] for i in range(len(df[column])-3*length,len(df[column]))]))  # Chnage df.loc with column_data[i]
            f.write(f'{country},{column},{selected_index},{prediction}, Average\n')
            counter_8 +=1
        # selected_weekly_column_data = [[index, value] for index, value in enumerate(df.loc[:, column]) if index in range(7*24*length,len(df[column])-7*24*length) and pd.isnull(value)]
        # counter3 += len(selected_weekly_column_data)
        # for selected_index,selected_value in selected_weekly_column_data:

            # if pd.isnull(column_data[selected_index]) and selected_index in range(7*24*length, len(df[column])-7*24*length):
            #     f.write(f'{country},{column},{selected_index},{(column_data[selected_index-7*24*length] + column_data[selected_index +7*24*length])/2} \n')
            # elif pd.isnull(column_data[selected_index]) and selected_index not in range(7*24*length, len(df[column])-7*24*length):
            #     f.write(f'{country},{column},{selected_index},{0} \n')

                # print(country, column, selected_index, selected_index-7*24*length, selected_index +7*24*length)
    # print(country,counter5)
for i in range(9):
    print(globals()[f'counter_{i}'])


0
0
0
0
0
0
0
0
0


## Column wise check what generation type suits hourly and which suits weekly

In [13]:
countries=['Belgium','Switzerland','Denmark','Finland','Latvia','Norway','Poland','Portugal','Slovakia','Slovenia']
# countries = ['Belgium']
f = open('neighbors of missing values.txt', 'a')
for country in countries:
    df_orig, length = ii.omit_dst(pd.read_csv(f'../Data Sources/ENTSO-E/2018/Load/{country}.csv', low_memory=False).iloc[:, 2:])
    if 'Hydro Pumped Storage  - Actual Consumption [MW]' in df_orig.columns.values:
        df_orig = df_orig.drop(['Hydro Pumped Storage  - Actual Consumption [MW]'], axis=1)
    df_hourly = copy.deepcopy(df_orig)
    df_weekly = copy.deepcopy(df_orig)

    for column in df_orig.columns.values:
        df_orig = df_orig.replace('n/e', 0)
        if(df_orig[column] == 0).all():
            df_orig = df_orig.drop(column, axis=1)
    
    for column in df_orig.columns.values:

        deleted_values = []
        values = np.random.randint(low=0, high=8760*length, size=500*length)
        for value in values:
            # x = random.choice(range(20))
            df_hourly.loc[value, column] = np.nan
            df_weekly.loc[value, column] = np.nan
            deleted_values.append([value, column])

        column_data = {}
        for index, value in enumerate(df_hourly.loc[:, column]):
            column_data[index] = value
        # selected_hourly_column_data = [[index, value] for index, value in enumerate(df.loc[:, column]) if index in range(3*length,len(df[column])-3*length) and pd.isnull(value)]
        # for selected_index,selected_value in selected_hourly_column_data:
        for selected_index in values:
            if selected_index in range(3*length, len(df_hourly[column])-3*length):
                selected_values = []
                for i in [x for x in range(-3, 4) if x != 0]:
                    selected_values.append(
                        [selected_index + i*length, column_data[selected_index + i*length]])
                if pd.isnull(selected_values).sum() >= 3 and selected_index in range(27*length, len(df_hourly[column])-27*length):
                    selected_values = []
                    for i in [x for x in range(-3, 4) if x != 0]:
                        for j in [-24, 0, 24]:
                            selected_values.append(
                                [selected_index + (i+j)*length, column_data[selected_index + (i+j)*length]])
                    if pd.isnull(selected_values).sum() >= 12 and selected_index in range(51*length, len(df_hourly[column])-51*length):
                        selected_values = []
                        for i in [x for x in range(-3, 4) if x != 0]:
                            for j in [-48, -24, 0, 24, 48]:
                                selected_values.append(
                                    [selected_index + (i+j)*length, column_data[selected_index + (i+j)*length]])
                        # print(selected_values)
                        if pd.isnull(selected_values).sum() >= 24 and pd.isnull(selected_values).sum() < len(selected_values):
                            prediction = mean([i[1] for i in selected_values])
                            df_hourly.loc[selected_index, column] = prediction
 
                        elif pd.isnull(selected_values).sum() < 24:
                            min_deg, prediction = polynomial(
                                selected_values, selected_index)
                            df_hourly.loc[selected_index, column] = prediction
                            

                        else:
                            df_hourly.loc[selected_index, column] = 0
                            
                    elif pd.isnull(selected_values).sum() >= 12:
                        prediction = mean([i[1] for i in selected_values])
                        df_hourly.loc[selected_index, column] = prediction
                       

                    else:
                        min_deg, prediction = polynomial(
                            selected_values, selected_index)
                        df_hourly.loc[selected_index, column] = prediction
                        
                elif pd.isnull(selected_values).sum() >= 3:
                    prediction = mean([i[1] for i in selected_values])
                    df_hourly.loc[selected_index, column] = prediction
                   
                else:
                    min_deg, prediction = polynomial(
                        selected_values, selected_index)
                    df_hourly.loc[selected_index, column] = prediction
                   

            elif selected_index < 3*length:
                selected_values = [column_data[i] for i in range(3*length)]
                prediction = mean(selected_values)
                df_hourly.loc[selected_index, column] = prediction

            elif selected_index >= (len(df_hourly[column])-3*length):
                selected_values = [column_data[i] for i in range(len(df_hourly[column])-3*length, len(df_hourly[column]))]
                prediction = mean(selected_values)
                df_hourly.loc[selected_index, column] = prediction

            if selected_index in range(7*24*length, len(df_weekly[column])-7*24*length):
                df_weekly.loc[selected_index, column] = (
                    column_data[selected_index-7*24*length] + column_data[selected_index + 7*24*length])/2

            elif selected_index not in range(7*24*length, len(df_weekly[column])-7*24*length):
                df_weekly.loc[selected_index, column] = 0

        # index = [i for i in range(8760*length)]
        counter_1 = [df_hourly.loc[i, column] for i in values]
        counter_1 = [0 if pd.isna(x) else x for x in counter_1]
        sum_orig = sum([df_orig.loc[i, column] for i in values])
        sum_edit = sum(counter_1)

        percentage = round((sum_orig - sum_edit)*100/sum_orig, 3)
        if percentage > 0:
            f.write(f'{country} - {column}: {percentage}\n')
            print(f'{country} - {column}: {percentage}')

f.close()


Belgium - Actual Total Load [MW] - Belgium (BE): 0.143
Denmark - Actual Total Load [MW] - Denmark (DK): 0.287
Finland - Actual Total Load [MW] - Finland (FI): 0.084
Norway - Actual Total Load [MW] - Norway (NO): 0.143
Poland - Actual Total Load [MW] - Poland (PL): 0.342
Slovakia - Actual Total Load [MW] - Slovakia (SK): 0.104
Slovenia - Actual Total Load [MW] - Slovenia (SI): 0.002


In [16]:
csvs = glob.glob("../Data Sources/ENTSO-E/2018/Transmission/*.csv")
tx_list = []
length =1
for csv in csvs:
    temp, divider_transmission = ii.omit_dst(pd.read_csv(csv).iloc[:,1:])
    if temp.isnull().sum().sum() == 0:
        tx_list.append(csv)

f = open('neighbors of missing values.txt', 'a')
for tx in tx_list:
    df_orig = pd.read_csv(tx, low_memory=False).iloc[:, 1:]
    if 'Hydro Pumped Storage  - Actual Consumption [MW]' in df_orig.columns.values:
        df_orig = df_orig.drop(
            ['Hydro Pumped Storage  - Actual Consumption [MW]'], axis=1)
    df_hourly = copy.deepcopy(df_orig)
    df_weekly = copy.deepcopy(df_orig)

    for column in df_orig.columns.values:
        df_orig = df_orig.replace('n/e', 0)
        if(df_orig[column] == 0).all():
            df_orig = df_orig.drop(column, axis=1)
    
    df_hourly = copy.deepcopy(df_orig)
    df_weekly = copy.deepcopy(df_orig)

    for column in df_orig.columns.values:

        deleted_values = []
        values = np.random.randint(low=0, high=8760*length, size=500*length)
        for value in values:
            # x = random.choice(range(20))
            df_hourly.loc[value, column] = np.nan
            df_weekly.loc[value, column] = np.nan
            deleted_values.append([value, column])

        column_data = {}
        for index, value in enumerate(df_hourly.loc[:, column]):
            column_data[index] = value
        # selected_hourly_column_data = [[index, value] for index, value in enumerate(df.loc[:, column]) if index in range(3*length,len(df[column])-3*length) and pd.isnull(value)]
        # for selected_index,selected_value in selected_hourly_column_data:
        for selected_index in values:
            if selected_index in range(3*length, len(df_hourly[column])-3*length):
                selected_values = []
                for i in [x for x in range(-3, 4) if x != 0]:
                    selected_values.append(
                        [selected_index + i*length, column_data[selected_index + i*length]])
                if pd.isnull(selected_values).sum() >= 3 and selected_index in range(27*length, len(df_hourly[column])-27*length):
                    selected_values = []
                    for i in [x for x in range(-3, 4) if x != 0]:
                        for j in [-24, 0, 24]:
                            selected_values.append(
                                [selected_index + (i+j)*length, column_data[selected_index + (i+j)*length]])
                    if pd.isnull(selected_values).sum() >= 12 and selected_index in range(51*length, len(df_hourly[column])-51*length):
                        selected_values = []
                        for i in [x for x in range(-3, 4) if x != 0]:
                            for j in [-48, -24, 0, 24, 48]:
                                selected_values.append(
                                    [selected_index + (i+j)*length, column_data[selected_index + (i+j)*length]])
                        # print(selected_values)
                        if pd.isnull(selected_values).sum() >= 24 and pd.isnull(selected_values).sum() < len(selected_values):
                            prediction = mean([i[1] for i in selected_values])
                            df_hourly.loc[selected_index, column] = prediction

                        elif pd.isnull(selected_values).sum() < 24:
                            min_deg, prediction = polynomial(
                                selected_values, selected_index)
                            df_hourly.loc[selected_index, column] = prediction
                            

                        else:
                            df_hourly.loc[selected_index, column] = 0
                            
                    elif pd.isnull(selected_values).sum() >= 12:
                        prediction = mean([i[1] for i in selected_values])
                        df_hourly.loc[selected_index, column] = prediction
                        

                    else:
                        min_deg, prediction = polynomial(
                            selected_values, selected_index)
                        df_hourly.loc[selected_index, column] = prediction
                        
                elif pd.isnull(selected_values).sum() >= 3:
                    prediction = mean([i[1] for i in selected_values])
                    df_hourly.loc[selected_index, column] = prediction
                    
                else:
                    min_deg, prediction = polynomial(
                        selected_values, selected_index)
                    df_hourly.loc[selected_index, column] = prediction
                    

            elif selected_index < 3*length:
                selected_values = [column_data[i] for i in range(3*length)]
                prediction = mean(selected_values)
                df_hourly.loc[selected_index, column] = prediction

            elif selected_index >= (len(df_hourly[column])-3*length):
                selected_values = [column_data[i] for i in range(len(df_hourly[column])-3*length, len(df_hourly[column]))]
                prediction = mean(selected_values)
                df_hourly.loc[selected_index, column] = prediction

        if selected_index in range(7*24*length, len(df_weekly[column])-7*24*length):
            df_weekly.loc[selected_index, column] = (
                column_data[selected_index-7*24*length] + column_data[selected_index + 7*24*length])/2

        elif selected_index not in range(7*24*length, len(df_weekly[column])-7*24*length):
            df_weekly.loc[selected_index, column] = 0

        # index = [i for i in range(8760*length)]
        counter_1 = [df_hourly.loc[i, column] for i in values]
        counter_1 = [0 if pd.isna(x) else x for x in counter_1]
        sum_orig = sum([df_orig.loc[i, column] for i in values])
        sum_edit = sum(counter_1)

        percentage = round((sum_orig - sum_edit)*100/sum_orig, 3)
        if percentage > 0:
            f.write(f'{tx[42:44]} - > {tx[45:47]} - {column}: {percentage}\n')
            print(f'{tx[42:44]} - > {tx[45:47]} - {column}: {percentage}')

f.close()


AT - > CZ - Czech Republic (CZ) > Austria (AT) [MW]: 0.15


C:\Users\lovin\AppData\Local\Temp/ipykernel_5848/735835783.py:117: RuntimeWarning: divide by zero encountered in double_scalars
  percentage = round((sum_orig - sum_edit)*100/sum_orig, 3)


AT - > DE - Germany (DE) > Austria (AT) [MW]: 0.515
AT - > DE - Austria (AT) > Germany (DE) [MW]: 0.184
BE - > LU - Belgium (BE) > Luxembourg (LU) [MW]: 1.039
BG - > GR - Greece (GR) > Bulgaria (BG) [MW]: 21.752
BG - > GR - Bulgaria (BG) > Greece (GR) [MW]: 0.411
CH - > DE - Germany (DE) > Switzerland (CH) [MW]: 0.439
CH - > DE - Switzerland (CH) > Germany (DE) [MW]: 4.158
CH - > IT - Italy (IT) > Switzerland (CH) [MW]: 8.882
CH - > IT - Switzerland (CH) > Italy (IT) [MW]: 0.672
CZ - > PL - Poland (PL) > Czech Republic (CZ) [MW]: 0.085


C:\Users\lovin\AppData\Local\Temp/ipykernel_5848/735835783.py:117: RuntimeWarning: invalid value encountered in double_scalars
  percentage = round((sum_orig - sum_edit)*100/sum_orig, 3)
C:\Users\lovin\AppData\Local\Temp/ipykernel_5848/735835783.py:117: RuntimeWarning: invalid value encountered in double_scalars
  percentage = round((sum_orig - sum_edit)*100/sum_orig, 3)
C:\Users\lovin\AppData\Local\Temp/ipykernel_5848/735835783.py:117: RuntimeWarning: invalid value encountered in double_scalars
  percentage = round((sum_orig - sum_edit)*100/sum_orig, 3)


DK - > NO - Norway (NO) > Denmark (DK) [MW]: 3.213
DK - > NO - Denmark (DK) > Norway (NO) [MW]: 0.707
ES - > FR - Spain (ES) > France (FR) [MW]: 3.632
ES - > PT - Portugal (PT) > Spain (ES) [MW]: 4.333
FI - > NO - Finland (FI) > Norway (NO) [MW]: 3.714
FI - > SE - Finland (FI) > Sweden (SE) [MW]: 3.071
FR - > IT - Italy (IT) > France (FR) [MW]: 24.331
HR - > BA - Bosnia and Herz. (BA) > Croatia (HR) [MW]: 0.079
HR - > HU - Croatia (HR) > Hungary (HU) [MW]: 0.315
HU - > RO - Romania (RO) > Hungary (HU) [MW]: 0.273
HU - > RO - Hungary (HU) > Romania (RO) [MW]: 0.967
HU - > SK - Slovakia (SK) > Hungary (HU) [MW]: 0.185
HU - > SK - Hungary (HU) > Slovakia (SK) [MW]: 14.434


...

C:\Users\lovin\AppData\Local\Temp/ipykernel_5848/735835783.py:117: RuntimeWarning: invalid value encountered in double_scalars
  percentage = round((sum_orig - sum_edit)*100/sum_orig, 3)


PL - > SK - Slovakia (SK) > Poland (PL) [MW]: 23.089
PL - > SK - Poland (PL) > Slovakia (SK) [MW]: 2.157
PL - > UA - Ukraine (UA) > Poland (PL) [MW]: 0.096


C:\Users\lovin\AppData\Local\Temp/ipykernel_5848/735835783.py:117: RuntimeWarning: invalid value encountered in double_scalars
  percentage = round((sum_orig - sum_edit)*100/sum_orig, 3)


RS - > BA - Serbia (RS) > Bosnia and Herz. (BA) [MW]: 1.055
RS - > MK - North Macedonia (MK) > Serbia (RS) [MW]: 0.794
RS - > MK - Serbia (RS) > North Macedonia (MK) [MW]: 1.525


C:\Users\lovin\AppData\Local\Temp/ipykernel_5848/735835783.py:117: RuntimeWarning: invalid value encountered in double_scalars
  percentage = round((sum_orig - sum_edit)*100/sum_orig, 3)
C:\Users\lovin\AppData\Local\Temp/ipykernel_5848/735835783.py:117: RuntimeWarning: invalid value encountered in double_scalars
  percentage = round((sum_orig - sum_edit)*100/sum_orig, 3)
